In [56]:
import re
from bs4 import BeautifulSoup
from pykml import parser
import pandas as pd
import json


In [57]:
def getAedInfo(kmlDescription,requiredFields):
    aedInfo = {}
    delimeters = '|'.join([f'({field})'for field in requiredFields])
    kmlDescription = re.split(delimeters,str(kmlDescription))
    kmlDescription = list(filter(lambda a: a != None, kmlDescription))
    for i,obj in enumerate(kmlDescription):
        if obj in requiredFields:
            eateryInfo[obj]=kmlDescription[i+1].split('</td>')[0].split('<td>')[-1]
    return eateryInfo
def clean_coordinates_col(coord):
    coord = coord.split(',')
    return f'{coord[1]},{coord[0]}'
def clean_name_col(NAME):
    return NAME.split('-')[0]
def clean_unit_col(UNIT):
    return UNIT.split(',')[0]

In [25]:
filename=r"C:\Users\tiahw\Documents\Projects\AedFinder\data\public-access-aed-locations-kml.kml"
with open(filename) as f:
    folder = parser.parse(f).getroot().Document.Folder

    
requiredFields = ['AED_ID','OPERATING_HOURS','HOUSE_NUMBER','ROAD_NAME','BUILDING_NAME','UNIT_NUMBER','POSTAL_CODE', 'AED_LOCATION_DESCRIPTION']

cordi=[]
db=pd.DataFrame()
for pm in folder.Placemark:
    plcs1=pm.Point.coordinates
    cordi.append(plcs1.text)
    db = db.append(getAedInfo(pm.description,requiredFields),ignore_index=True)

db['coordinates'] = cordi
db['coordinates'] = db.coordinates.apply(clean_coordinates_col)
# db['NAME'] = db.NAME.apply(clean_name_col)
# db['ADDRESSUNITNUMBER'] = db.ADDRESSUNITNUMBER.apply(clean_unit_col)
# db.rename(columns = {'NAME':'name'},inplace=True)
# db['address'] = db.ADDRESSBUILDINGNAME+' '+db.ADDRESSSTREETNAME+' #'+db.ADDRESSFLOORNUMBER+'-'+db.ADDRESSUNITNUMBER+' Singapore '+db.ADDRESSPOSTALCODE

In [58]:
db[['LATITUDE','LONGITUDE']] = db['coordinates'].str.split(',',1,expand =True)

In [59]:
relevant_col = ['AED_ID','OPERATING_HOURS','HOUSE_NUMBER','ROAD_NAME','BUILDING_NAME','UNIT_NUMBER','POSTAL_CODE', 'AED_LOCATION_DESCRIPTION','LATITUDE','LONGITUDE']

In [60]:
json_data = {'aed':list(db[relevant_col].T.to_dict().values())};
json_data

{'aed': [{'AED_ID': '529905-003',
   'OPERATING_HOURS': 'Mon - Sun 00:00-23:59;',
   'HOUSE_NUMBER': '6A',
   'ROAD_NAME': 'Simei Street 3',
   'BUILDING_NAME': 'Changi General Hospital',
   'UNIT_NUMBER': '',
   'POSTAL_CODE': '529905',
   'AED_LOCATION_DESCRIPTION': 'Level 1 Near Visitor Registration Counter',
   'LATITUDE': '1.34124667578049',
   'LONGITUDE': '103.948257779835'},
  {'AED_ID': '529905-001',
   'OPERATING_HOURS': 'Mon - Sun 00:00-23:59;',
   'HOUSE_NUMBER': '6A',
   'ROAD_NAME': 'Simei Street 3',
   'BUILDING_NAME': 'Changi General Hospital',
   'UNIT_NUMBER': '',
   'POSTAL_CODE': '529905',
   'AED_LOCATION_DESCRIPTION': 'Level B1 Lift Lobby J',
   'LATITUDE': '1.34124667578049',
   'LONGITUDE': '103.948257779835'},
  {'AED_ID': '529905-002',
   'OPERATING_HOURS': 'Mon - Sun 00:00-23:59;',
   'HOUSE_NUMBER': '6A',
   'ROAD_NAME': 'Simei Street 3',
   'BUILDING_NAME': 'Changi General Hospital',
   'UNIT_NUMBER': '',
   'POSTAL_CODE': '529905',
   'AED_LOCATION_DESCRIP

In [61]:
json_object = json.dumps(json_data, indent = 4)

In [62]:
with open(r"C:\Users\tiahw\Documents\Projects\AedFinder\data\Aed.json", "w") as outfile:
    outfile.write(json_object)